<a href="https://colab.research.google.com/github/yamini4336/prompt-based-ABSA-project/blob/main/absa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving laptop.csv to laptop.csv


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from huggingface_hub import whoami
whoami()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'type': 'user',
 'id': '693dc261eafc212cace1ac98',
 'name': 'yamini-morasa25',
 'fullname': 'Yamini Morasa',
 'email': 'yaminimorasa@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'billingMode': 'prepaid',
 'periodEnd': 1769904000,
 'isPro': False,
 'avatarUrl': '/avatars/3071044917aa9b9b216ca10e51c6652c.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Yamini Morasa',
   'role': 'read',
   'createdAt': '2025-12-27T16:06:13.223Z'}}}

In [ ]:
!pip install -U bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.4 MB/s eta 0:00:00


In [ ]:
# ============================================================
# FINAL GATED HYBRID ABSA (COLAB T4 SAFE)
# DeBERTa-v3-base + Gemma-2B (8-bit, frozen)
# ============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# -------------------------
# CONFIG (SAFE LIMITS)
# -------------------------
DEVICE = "cuda"
MAX_LEN = 96
BATCH_SIZE = 2
EPOCHS = 4
LR = 2e-5

label2id = {"negative": 0, "neutral": 1, "positive": 2}

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("/content/laptop.csv")
df = df.rename(columns={
    "Sentence": "sentence",
    "Aspect Term": "aspect",
    "polarity": "label"
})
df["label"] = df["label"].str.lower().str.strip()
df = df[df["label"].isin(label2id)].reset_index(drop=True)

train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=42
)

# -------------------------
# PROMPT FOR GEMMA
# -------------------------
def build_prompt(sentence, aspect):
    return (
        f"Sentence: {sentence}\n"
        f"Aspect: {aspect}\n"
        f"What is the sentiment towards the aspect?"
    )

# -------------------------
# TOKENIZERS
# -------------------------
deberta_tok = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

bnb_cfg = BitsAndBytesConfig(load_in_8bit=True)
gemma_tok = AutoTokenizer.from_pretrained("google/gemma-2b")

gemma = AutoModel.from_pretrained(
    "google/gemma-2b",
    quantization_config=bnb_cfg,
    device_map="auto"
)
for p in gemma.parameters():
    p.requires_grad = False

# -------------------------
# DATASET
# -------------------------
class ABSADataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]

        deb = deberta_tok(
            r.sentence, r.aspect,
            max_length=MAX_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        gem = gemma_tok(
            build_prompt(r.sentence, r.aspect),
            max_length=MAX_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "ids": deb["input_ids"].squeeze(),
            "mask": deb["attention_mask"].squeeze(),
            "g_ids": gem["input_ids"].squeeze(),
            "g_mask": gem["attention_mask"].squeeze(),
            "label": torch.tensor(label2id[r.label])
        }

train_loader = DataLoader(
    ABSADataset(train_df),
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    ABSADataset(val_df),
    batch_size=BATCH_SIZE
)

# -------------------------
# MODEL (GATED FUSION)
# -------------------------
class HybridABSA(nn.Module):
    def __init__(self):
        super().__init__()

        # DeBERTa encoder
        self.deberta = AutoModel.from_pretrained("microsoft/deberta-v3-base")

        self.attn = nn.MultiheadAttention(
            embed_dim=768, num_heads=8, batch_first=True
        )

        # Project Gemma → DeBERTa space
        self.gemma_proj = nn.Linear(2048, 768)

        # Gating network
        self.gate = nn.Sequential(
            nn.Linear(768 * 2, 1),
            nn.Sigmoid()
        )

        self.classifier = nn.Linear(768, 3)

    def forward(self, ids, mask, g_ids, g_mask):
        # ---- DeBERTa ----
        d = self.deberta(
            input_ids=ids,
            attention_mask=mask
        ).last_hidden_state

        d, _ = self.attn(d, d, d)
        d = d.mean(dim=1)  # (B, 768)

        # ---- Gemma (frozen) ----
        with torch.no_grad():
            g = gemma(
                input_ids=g_ids,
                attention_mask=g_mask
            ).last_hidden_state.mean(dim=1)  # (B, 2048)

        g = self.gemma_proj(g.float())

        # (B, 768)

        # ---- GATED FUSION ----
        gate_val = self.gate(torch.cat([d, g], dim=1))  # (B, 1)
        fused = gate_val * g + (1 - gate_val) * d

        return self.classifier(fused)

model = HybridABSA().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# -------------------------
# TRAIN / EVAL
# -------------------------
def run_epoch(loader, train=True):
    model.train() if train else model.eval()
    preds, gold = [], []

    for b in loader:
        ids = b["ids"].to(DEVICE)
        mask = b["mask"].to(DEVICE)
        g_ids = b["g_ids"].to(DEVICE)
        g_mask = b["g_mask"].to(DEVICE)
        y = b["label"].to(DEVICE)

        if train:
            optimizer.zero_grad()

        logits = model(ids, mask, g_ids, g_mask)
        loss = F.cross_entropy(logits, y)

        if train:
            loss.backward()
            optimizer.step()

        preds.extend(logits.argmax(1).cpu().tolist())
        gold.extend(y.cpu().tolist())

    return accuracy_score(gold, preds), f1_score(gold, preds, average="macro")

# -------------------------
# TRAINING LOOP
# -------------------------
for epoch in range(EPOCHS):
    tr_acc, tr_f1 = run_epoch(train_loader, True)
    va_acc, va_f1 = run_epoch(val_loader, False)

    print(
        f"Epoch {epoch+1} | "
        f"Train Acc {tr_acc:.4f}, F1 {tr_f1:.4f} | "
        f"Val Acc {va_acc:.4f}, F1 {va_f1:.4f}"
    )


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 | Train Acc 0.6957, F1 0.6078 | Val Acc 0.7927, F1 0.7473
Epoch 2 | Train Acc 0.8211, F1 0.7789 | Val Acc 0.7948, F1 0.7369
Epoch 3 | Train Acc 0.8735, F1 0.8439 | Val Acc 0.8099, F1 0.7803
Epoch 4 | Train Acc 0.9178, F1 0.9021 | Val Acc 0.8207, F1 0.7981


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["label"],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=42
)


In [ ]:
test_loader = DataLoader(
    ABSADataset(test_df),
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

def evaluate_test(model, loader):
    model.eval()
    preds, gold = [], []

    with torch.no_grad():
        for b in loader:
            ids = b["ids"].to(DEVICE)
            mask = b["mask"].to(DEVICE)

            # ---- if using LLM model ----
            if "g_ids" in b:
                g_ids = b["g_ids"].to(DEVICE)
                g_mask = b["g_mask"].to(DEVICE)
                logits = model(ids, mask, g_ids, g_mask)
            # ---- if using syntax-GCN model ----
            else:
                adj = b["adj"].to(DEVICE)
                logits = model(ids, mask, adj)

            y = b["label"].to(DEVICE)

            preds.extend(logits.argmax(1).cpu().tolist())
            gold.extend(y.cpu().tolist())

    acc = accuracy_score(gold, preds)
    f1  = f1_score(gold, preds, average="macro")

    print("FINAL TEST RESULTS")
    print("Test Accuracy :", round(acc, 4))
    print("Test Macro-F1 :", round(f1, 4))
    print("\nClassification Report:\n")
    print(classification_report(gold, preds, digits=4))

    return acc, f1


# run test evaluation
test_acc, test_f1 = evaluate_test(model, test_loader)


FINAL TEST RESULTS
Test Accuracy : 0.8646
Test Macro-F1 : 0.8464

Classification Report:

              precision    recall  f1-score   support

           0     0.9391    0.8308    0.8816       130
           1     0.7143    0.7971    0.7534        69
           2     0.8839    0.9257    0.9043       148

    accuracy                         0.8646       347
   macro avg     0.8458    0.8512    0.8464       347
weighted avg     0.8709    0.8646    0.8658       347



In [ ]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
def predict_sentiment(sentence, aspect, model):
    model.eval()

    # ---- DeBERTa input ----
    deb = deberta_tok(
        sentence, aspect,
        max_length=MAX_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # ---- Gemma prompt input ----
    gem = gemma_tok(
        build_prompt(sentence, aspect),
        max_length=MAX_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    with torch.no_grad():
        logits = model(
            ids=deb["input_ids"].to(DEVICE),
            mask=deb["attention_mask"].to(DEVICE),
            g_ids=gem["input_ids"].to(DEVICE),
            g_mask=gem["attention_mask"].to(DEVICE)
        )

        pred_id = torch.argmax(logits, dim=1).item()

    return id2label[pred_id]

In [ ]:
sentence = "The battery life is great but the screen is dull."
aspect = "battery life"

prediction = predict_sentiment(sentence, aspect, model)

print("Sentence:", sentence)
print("Aspect:", aspect)
print("Predicted Sentiment:", prediction)

Sentence: The battery life is great but the screen is dull.
Aspect: battery life
Predicted Sentiment: positive


In [ ]:
sentence = "The battery life is great but the screen is dull."
aspects = ["battery life", "screen"]

for asp in aspects:
    pred = predict_sentiment(sentence, asp, model)
    print(f"Aspect: {asp} → {pred}")

Aspect: battery life → positive
Aspect: screen → negative
